In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 47.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=84bfeba00a0eff0095119ab6c1a0ad319b5932e34756131415c74c50997f8dbe
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


## Создаём точку входа в Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.

## Считывание данных
Скачиваем и копируем в папку с нотебуком
https://www.kaggle.com/iabhishekofficial/mobile-price-classification

Внутри содержится следующая информация:

* id: ID
* battery_power: Общая энергия, которую батарея может накопить за один раз (мАч)
* blue: Поддержка Bluetooth или нет
* clock_speed: Скорость, с которой микропроцессор выполняет инструкции
* dual_sim: Поддержка двух sim-карт или нет
* fc: Мегапиксели фронтальной камеры
* four_g: Поддержка 4G или нет
* int_memory: Внутренняя память (ГБ)
* m_dep: Подвижная глубина (см)
* mobile_wt: Вес мобильного телефона
* n_cores: Количество ядер процессора
* pc: Мегапиксели основной камеры
* px_height: Высота Разрешения в Пикселях
* px_width: Ширина Разрешения в Пикселях
* ram: Оперативная память (МБ)
* sc_h: Высота экрана мобильного устройства (см)
* sc_w: Ширина экрана мобильного устройства (см)
* talk_time: Время разговора на одном заряде батареи
* three_g: Поддержка 3G или нет
* touch_screen: Есть сенсорный экран или нет
* wifi: Поддержка Wi-Fi или нет

In [4]:
train = sc.textFile('/content/train.csv')
test = sc.textFile('/content/test.csv')

In [5]:
# Вывести содержимое каждого файла (часть строк)
train.take(2)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '769,1,2.9,1,0,0,9,0.1,182,5,1,248,874,3946,5,0,7,0,0,0,3']

In [6]:
test.take(2)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0']

##  Преобразуем train и test

In [7]:
# Преобразуем train
train_first_row = train.first()

train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[:-1]])

In [8]:
# Преобразуем test
test_first_row = test.first()

test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[1:]])

## Объединим train и test

In [9]:
data = train.union(test)
data_first_row = train_first_row.split(',')

In [ ]:
data.take(1)

In [ ]:
# Объединим train и test
#import pandas as pd
#data = pd.concat(map(pd.read_csv, ['train.csv', 'test.csv']))

## Рассчитайте соотношение сторон телефона и экрана

In [54]:
# Выведите самое популярное распределение соотношений сторон экрана
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
col_height = data_first_row.index('px_height')
col_width = data_first_row.index('px_width')
data.map(lambda row: (row[col_width], row[col_height])).filter(lambda row: row[1]!=0)\
.map(lambda row: (('>= 16:9' if row[0]/row[1] >= 1.77 else '< 16:9' ), 1)).reduceByKey(lambda x, y: x + y).sortByKey().collect()

[('< 16:9', 1353), ('>= 16:9', 1643)]

In [55]:
# Выведите самые популярные распределение плотности пикселей (точек на дюйм)
import math
col_sc_h = data_first_row.index('sc_h')
col_sc_w = data_first_row.index('sc_w')
data\
.map(lambda row: (round((math.sqrt(row[col_width]**2 + row[col_height]**2))/(math.sqrt(row[col_sc_h]**2 + row[col_sc_w]**2))),1))\
.reduceByKey(lambda x, y: x + y).map(lambda x: (x[1], x[0])).sortByKey(False).take(10)

[(35, 82),
 (34, 54),
 (31, 78),
 (28, 84),
 (28, 71),
 (28, 79),
 (27, 81),
 (27, 89),
 (27, 65),
 (27, 90)]

## JOIN

In [65]:
# По соотношению сторон
train_inner = train.map(lambda row: (round((math.sqrt(row[col_width]**2 + row[col_height]**2) ) / (math.sqrt(row[col_sc_h]**2 + row[col_sc_w]**2))), 1))
test_inner = test.map(lambda row: (round((math.sqrt(row[col_width]**2 + row[col_height]**2) ) / (math.sqrt(row[col_sc_h]**2 + row[col_sc_w]**2))), 1))

In [69]:
train_inner_px = train.map(lambda row: (row[col_width], row[col_height])).filter(lambda row: row[1]!=0).map(lambda row: (('>= 16:9' if row[0]/row[1] >= 1.77 else '< 16:9' ), 1))
test_inner_px = test.map(lambda row: (row[col_width], row[col_height])).filter(lambda row: row[1]!=0).map(lambda row: (('>= 16:9' if row[0]/row[1] >= 1.77 else '< 16:9' ), 1))
inner_data_px = train_inner_px.cogroup(test_inner_px)
inner_data_px.map(lambda row: (row[0], sum([len(el) for el in row[1]]) )).sortByKey(True).collect()

[('< 16:9', 1353), ('>= 16:9', 1643)]

In [67]:
# По плотности пикселей
inner_data_rate = train_inner.cogroup(test_inner)

In [70]:
inner_data_rate.map(lambda row: (row[0], sum([len(el) for el in row[1]]) )).map(lambda x: (x[1], x[0])).sortByKey(False).take(10)

[(35, 82),
 (34, 54),
 (31, 78),
 (28, 84),
 (28, 71),
 (28, 79),
 (27, 81),
 (27, 89),
 (27, 65),
 (27, 90)]

## DataFrame

In [77]:
# Считываем данные
df_train = spark.read.csv('train.csv', header=True, inferSchema=True)
df_test = spark.read.csv('test.csv', header=True, inferSchema=True)

In [81]:
# Объединяем
data = df_test.unionByName(df_train, allowMissingColumns=True)

In [82]:
# Создаем колонки с соотношением сторон и плотностью пикселей
from pyspark.sql.functions import sqrt, round, col, pow
data = data.withColumn('asc_rate', round((col('px_width') / col('px_height')), 2))\
.withColumn('ppi', round((sqrt(pow(data.px_width, 2) + pow(data.px_height,2)) )/(sqrt(pow(data.sc_h, 2) + pow(data.sc_w, 2)))))

## Сохранение


In [84]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора на одном заряде батареи
data_save = data.select('id', 'ppi', 'talk_time')

In [85]:
data_save.write.csv('result.csv', sep=';', header=True, encoding='cp1251')